In [ ]:
filter_by_fqn: str = ""
filter_by_method_name: str = ""

In [ ]:
import iplantuml
from code_data_science import data_table as dt

df = dt.read_csv("../samples/call_graph.csv")

if filter_by_fqn:
    df = df[
        (df["fromClass"].str.contains(filter_by_fqn))
        | (df["toClass"].str.contains(filter_by_fqn))
    ]

In [ ]:
plantuml_code = ["@startuml"]
# TODO: add dynamic coloring based on actions or something in the future
colors = ["black", "red", "blue", "purple"]

classes = {}


def is_valid_class(class_name):
    if filter_by_fqn:
        return (
            True
            if isinstance(class_name, str) and class_name == filter_by_fqn
            else False
        )

    return True


def is_valid_method(method_name):
    if filter_by_method_name:
        return (
            True
            if isinstance(method_name, str) and method_name == filter_by_method_name
            else False
        )

    return True


def make_method(name):
    # return f"{name}(arguments)" if isinstance(arguments, str) else f"{name}()"
    # overloads are not supported in v1.
    return f"{name}()"


for i, row in df.iterrows():
    from_class = row["fromClass"]
    to_class = row["toClass"]
    if str(from_class).__contains__("$") or str(to_class).__contains__("$"):
        continue

    from_name = row["fromName"]
    to_name = row["toName"]

    if is_valid_class(from_class):
        if is_valid_method(from_name):
            if from_class not in classes:
                classes[from_class] = set()

            members = classes[from_class]
            method_name = make_method(from_name)
            if method_name not in members:
                members.add(make_method(from_name))

            if to_class not in classes:
                classes[to_class] = set()

            members = classes[to_class]
            method_name = make_method(to_name)
            if method_name not in members:
                members.add(make_method(to_name))

    if is_valid_class(to_class):
        if is_valid_method(to_name):
            if to_class not in classes:
                classes[to_class] = set()

            members = classes[to_class]
            method_name = make_method(to_name)
            if method_name not in members:
                members.add(make_method(to_name))

            if from_class not in classes:
                classes[from_class] = set()

            members = classes[from_class]
            method_name = make_method(from_name)
            if method_name not in members:
                members.add(make_method(from_name))

# Create classes
for _class in classes:
    plantuml_code.append(f"class {_class}")
    plantuml_code.append("{")
    for member in classes[_class]:
        plantuml_code.append(f"    +{member}")
    plantuml_code.append("}")

# Create relationships
for i, row in df.iterrows():
    from_class = row["fromClass"]
    to_class = row["toClass"]
    if str(from_class).__contains__("$") or str(to_class).__contains__("$"):
        continue
    from_name = row["fromName"]
    to_name = row["toName"]

    if is_valid_class(from_class):
        if is_valid_method(from_name):
            from_method = make_method(from_name)
            to_method = make_method(to_name)
            plantuml_code.append(
                f'"{from_class}::{from_method}" --> "{to_class}::{to_method}"'
            )

    if from_class != to_class and is_valid_class(to_class):
        if is_valid_method(to_name):
            from_method = make_method(to_name)
            to_method = make_method(to_name)
            plantuml_code.append(
                f'"{to_class}::{from_method}" <-- "{from_class}::{to_method}"'
            )

# needed for whatever reason plantuml needs it for when it needs it ¯\_(ツ)_/¯
plantuml_code.append("")
plantuml_code.append("@enduml")

plantuml_code_string = "\n".join(plantuml_code)

# print(plantuml_code_string)

In [ ]:
cell_content = f"%%plantuml\n{plantuml_code_string}"
get_ipython().run_cell(cell_content)